In [8]:
import numpy as np
import h5py
import pandas as pd

# Response quality index - QI
SD1: For each cluster, calculate the standard deviation (STD) of the firing rate for each trial in the ON-OFF pattern (lasting 2 seconds). Then, take the average of these STDs across all trials to get SD1.

SD2: Compute the average firing rate curve across all trials for the cluster. Then, calculate STD of this average firing rate curve. This value is called SD2.

Ratio: The ratio is then calculated by dividing SD1 by SD2.

In [11]:
def get_light_stimulus_info(h5_file_path):
    # Load the light stimulus timestamps
    with h5py.File(h5_file_path, 'r') as h5_file:
        light_stimulus_timestamps = h5_file['Data/Recording_0/EventStream/Stream_0/EventEntity_0'][:][0]
    
    # Convert timestamps from microseconds to seconds
    light_stimulus_timestamps_sec = light_stimulus_timestamps / 1_000_000.0
    return light_stimulus_timestamps_sec

def calculate_firing_rate_statistics(spike_times_file, spike_clusters_file, h5_file_path, bin_size=0.128):
    # Load spike data
    spike_times = np.load(spike_times_file)
    spike_clusters = np.load(spike_clusters_file)
    
    # Convert spike times to seconds, assuming a sampling rate of 50kHz
    sampling_rate = 50000
    spike_times_sec = spike_times / sampling_rate
    
    # Get light stimulus timestamps in seconds
    light_stimulus_timestamps_sec = get_light_stimulus_info(h5_file_path)
    
    # Define the start and end points for the snippets
    snippet_intervals = [(light_stimulus_timestamps_sec[i], light_stimulus_timestamps_sec[i + 2]) 
                         for i in range(0, 29, 2)]
    
    # Get all unique cluster IDs
    cluster_ids = np.unique(spike_clusters)
    
    # Initialize results storage
    results = []

    # Iterate through each cluster and calculate the statistics
    for cluster_id in cluster_ids:
        snippet_firing_rates = []
        for start_time, end_time in snippet_intervals:
            bins = np.arange(start_time, end_time + bin_size, bin_size)
            cluster_spike_times = spike_times_sec[(spike_clusters == cluster_id) & 
                                                  (spike_times_sec >= start_time) & 
                                                  (spike_times_sec <= end_time)]
            spike_counts, _ = np.histogram(cluster_spike_times, bins)
            firing_rate = spike_counts / bin_size
            snippet_firing_rates.append(firing_rate)
        
        snippet_firing_rates = np.array(snippet_firing_rates)
        
        # Calculate SD1: Average of STDs of all snippets
        snippet_stds = np.std(snippet_firing_rates, axis=1)
        sd1 = np.mean(snippet_stds)
        
        # Calculate SD2: STD of the overall average firing rate curve
        mean_firing_rate_curve = np.mean(snippet_firing_rates, axis=0)
        sd2 = np.std(mean_firing_rate_curve)
        
        # Calculate Ratio
        ratio = sd2 / sd1 if sd1 != 0 else np.nan
        
        # Store results
        results.append({
            'Cluster ID': cluster_id,
            'SD1 ': sd1,
            'SD2 ': sd2,
            'QI Ratio': ratio
        })

    # Convert results to a DataFrame for better visualization
    results_df = pd.DataFrame(results)
    print(results_df)
    return results_df

# Example usage
spike_times_file = r'C:\Users\Xinyi\Desktop\mr1L_dorsal_1v\Sorting\mr1L_dorsal_1v\mr1L_dorsal_1v\mr1L_dorsal_1v.GUI\spike_times.npy'  # Replace with your spike_times.npy file path
spike_clusters_file = r'C:\Users\Xinyi\Desktop\mr1L_dorsal_1v\Sorting\mr1L_dorsal_1v\mr1L_dorsal_1v\mr1L_dorsal_1v.GUI\spike_clusters.npy'  # Replace with your spike_clusters.npy file path
h5_file_path = r'C:\Users\Xinyi\Desktop\mr1L_dorsal_1v\Sorting\mr1L_dorsal_1v\mr1L_dorsal_1v.h5'  # Replace with your .h5 file path

df = calculate_firing_rate_statistics(spike_times_file, spike_clusters_file, h5_file_path)
df.to_csv('firing_rate_statistics.csv', index=False)


    Cluster ID       SD1        SD2   QI Ratio
0            0  26.544554  24.087288  0.907429
1            1  54.802255  51.945292  0.947868
2            2  42.631143  36.702528  0.860932
3            3  55.929789  53.374608  0.954314
4            4  33.938789  29.621591  0.872795
5            5  32.451397  31.463571  0.969560
6            6   9.099972   3.588267  0.394316
7            8  34.979040  31.615095  0.903830
8            9  18.643017  10.908348  0.585117
9           10  70.887742  67.286463  0.949197
10          11  77.809933  72.431187  0.930873
11          12  55.505921  52.847864  0.952112
12          13  57.670215  55.113513  0.955667
13          14  21.712934  17.009484  0.783380
14          15  50.297812  47.709184  0.948534
15          16  50.687603  46.109644  0.909683
16          17  18.806810  13.131004  0.698205
17          18  70.768944  67.908358  0.959579
18          19  83.878889  80.554141  0.960363
19          20  77.794588  76.688777  0.985786
20          2

# Photon isomerization
The method is based on https://github.com/eulerlab/mouse-scene-cam/blob/master/photoisomerization/cam_images_2_photoisomerization_v0_2.ipynb
Our example is ex-vivo model which has no attenuation factors and transmission of mouse optical aparatus.

In [16]:
P_el = 0.34e-6  # Electrical power in Watts (0.34 μW)
lambda_nm = 590  # Wavelength in nm
a = 6.242e18  # Conversion constant (photons/J)
c = 299792458  # Speed of light in m/s
h = 4.135667e-15  # Planck's constant in eV·s

# Ex vivo model: no attenuation in optical paths
mu_lens2cam = 1  # Attenuation factor

# Step 1: Calculate photon flux (P_Phi)
P_Phi = (P_el * a * lambda_nm * 1e-9) / (c * h) * (1 / mu_lens2cam)

# Constants for Step 2
A_Stim = 1e8  # Stimulated area in μm^2
A_Collect_M_cone = 0.2  # Collection area for M-cone in μm^2
A_Collect_rod = 0.5  # Collection area for rod in μm^2
S_Act = 0.858  # Sensitivity correction factor

# Step 2: Calculate photoisomerisation rates
R_Iso_M_cone = (P_Phi / A_Stim) * A_Collect_M_cone * S_Act
R_Iso_rod = (P_Phi / A_Stim) * A_Collect_rod * S_Act

# Results
print(f"Photon flux (P_Phi): {P_Phi:.2e} photons/s")
print(f"Photoisomerisation rate for M-cone (R_Iso_M_cone): {R_Iso_M_cone:.2f} P*/cone/s")
print(f"Photoisomerisation rate for rod (R_Iso_rod): {R_Iso_rod:.2f} P*/rod/s")


Photon flux (P_Phi): 1.01e+12 photons/s
Photoisomerisation rate for M-cone (R_Iso_M_cone): 1733.03 P*/cone/s
Photoisomerisation rate for rod (R_Iso_rod): 4332.57 P*/rod/s
